# groundup_004_forward_backward_passes

In [1]:
#| default_exp delete0004

In [2]:
#| export
a = "to delete"

## imports

In [3]:
from fastdebug.utils import *

In [4]:
from fastdebug.groundup import *

In [5]:
%whos function

Variable                    Type        Data/Info
-------------------------------------------------
check_data_directories      function    <function check_data_dire<...>tories at 0x7f99547f73a0>
chunks                      function    <function chunks at 0x7f99261531f0>
chunks_faster               function    <function chunks_faster at 0x7f9926153280>
fastcodes                   function    <function fastcodes at 0x7f99547f48b0>
fastlistnbs                 function    <function fastlistnbs at 0x7f99547f49d0>
fastlistsrcs                function    <function fastlistsrcs at 0x7f99547f4a60>
fastnbs                     function    <function fastnbs at 0x7f99547f4820>
fastnotes                   function    <function fastnotes at 0x7f99547f4940>
fastsrcs                    function    <function fastsrcs at 0x7f99547f4310>
fastview                    function    <function fastview at 0x7f99547df280>
get_all_nbs                 function    <function get_all_nbs at 0x7f99547f44c0>
get_exp_da

In [6]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor

In [7]:
mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
np.set_printoptions(precision=2, linewidth=140)

## get_exp_data, map, tensor

In [8]:
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'

In [9]:
Path

pathlib.Path

In [10]:
x_train, y_train, x_valid, y_valid = get_exp_data()

In [11]:
x_train.shape
type(x_train)

(50000, 784)

numpy.ndarray

In [12]:
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

In [13]:
x_train.shape
type(x_train)

torch.Size([50000, 784])

torch.Tensor

## Foundations version

### Basic architecture

In [14]:
# n,m = x_train.shape
# c = y_train.max()+1
# n,m,c

In [15]:
type(x_train)

torch.Tensor

In [16]:
r,c = x_train.shape
l = y_train.max()+1
r,c,l

(50000, 784, tensor(10))

In [17]:
# num hidden activations
nh = 50

In [18]:
# w1 = torch.randn(m,nh)
# b1 = torch.zeros(nh)
# w2 = torch.randn(nh,1)
# b2 = torch.zeros(1)

In [19]:
w1 = torch.randn(c,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)
w1.shape
b1.shape
w2.shape
b2.shape

torch.Size([784, 50])

torch.Size([50])

torch.Size([50, 1])

torch.Size([1])

In [20]:
#| export groundup
def lin(x, w, b): 
    "use torch.matmul (faster version of einsum) to create a linear model"
    return x@w + b

In [21]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [22]:
#| export groundup
def relu(x): 
    "basic relu with max in torch"
    return x.clamp_min(0.)

In [23]:
t = relu(lin(x_valid, w1, b1)) # add relu unto the linear model
t

tensor([[0.00, 0.00, 2.25,  ..., 0.00, 9.30, 0.00],
        [0.00, 0.00, 0.00,  ..., 0.00, 8.57, 0.00],
        [0.00, 2.04, 0.00,  ..., 0.00, 5.76, 0.00],
        ...,
        [0.00, 0.00, 0.00,  ..., 0.00, 0.00, 3.62],
        [0.00, 0.73, 0.00,  ..., 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00,  ..., 0.00, 0.00, 0.00]])

In [24]:
(t >= 0).shape
(t >= 0).count_nonzero()
(t < 0).count_nonzero()

torch.Size([10000, 50])

tensor(500000)

tensor(0)

In [25]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [26]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

### Loss function: MSE

We need to get rid of that trailing (,1), in order to use `mse`.

In [27]:
res[:,0].shape

torch.Size([10000])

(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [28]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

In [29]:
y_train,y_valid = y_train.float(),y_valid.float()

In [30]:
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [31]:
mse(preds, y_train)

tensor(6871.75)

### Gradients and backward pass

In [33]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [34]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [35]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    diff = out[:,0]-targ
    loss = res.pow(2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [36]:
forward_and_backward(x_train, y_train)

In [37]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [38]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [39]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [40]:
loss = forward(xt2, y_train)
loss.backward()

In [41]:
from fastcore.test import test_close

In [42]:
test_close(w22.grad, w2g, eps=0.01)
test_close(b22.grad, b2g, eps=0.01)
test_close(w12.grad, w1g, eps=0.01)
test_close(b12.grad, b1g, eps=0.01)
test_close(xt2.grad, ig , eps=0.01)

## Refactor model

### Layers as classes

In [43]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [44]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [45]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [46]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [47]:
model = Model(w1, b1, w2, b2)

In [48]:
%time loss = model(x_train, y_train)

CPU times: user 326 ms, sys: 3.48 ms, total: 329 ms
Wall time: 56.3 ms


In [49]:
%time model.backward()

CPU times: user 780 ms, sys: 206 ms, total: 986 ms
Wall time: 171 ms


In [50]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Module.forward()

In [51]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def bwd(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [52]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [53]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [54]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [55]:
model = Model(w1, b1, w2, b2)

In [56]:
%time loss = model(x_train, y_train)

CPU times: user 167 ms, sys: 0 ns, total: 167 ms
Wall time: 29.9 ms


In [57]:
%time model.backward()

CPU times: user 370 ms, sys: 287 ms, total: 657 ms
Wall time: 114 ms


In [58]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd

In [59]:
from torch import nn
import torch.nn.functional as F

In [60]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [61]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [62]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

NameError: name 'm' is not defined

In [ ]:
l0 = model.layers[0]
l0.b.grad